In [1]:
%cd hotpotqa_runs
%load_ext autoreload
%autoreload 2


/home/juggernautjha/Desktop/ayatollah-gaslighting/hotpotqa_runs


In [2]:
%env OPENAI_API_KEY=sk-sIFKZY2XuOmolhy6agIET3BlbkFJNOQhERMC9bGAma

env: OPENAI_API_KEY=sk-sIFKZY2XuOmolhy6agIET3BlbkFJNOQhERMC9bGAma


In [4]:
z = dict([('model_name', 'llama2'), ('is_ollama', True)])

In [8]:
import agents

In [9]:
import joblib
import numpy as np
import os, sys
sys.path.append("/content/reflexion/hotpotqa_runs")
from agents import CoTAgent, ReflexionStrategy
from util import summarize_trial, log_trial, save_agents

{'model_name': 'llama2', 'is_ollama': True, 'temperature': 0, 'max_tokens': 100, 'stop': '\n'}
{'model_name': 'llama2', 'is_ollama': True, 'temperature': 0, 'max_tokens': 100, 'stop': '\n'}
{'model_name': 'llama2', 'is_ollama': True, 'temperature': 0, 'max_tokens': 100, 'stop': '\n'}
{'model_name': 'llama2', 'is_ollama': True, 'temperature': 0, 'max_tokens': 100, 'stop': '\n'}
{'model_name': 'llama2', 'is_ollama': True, 'temperature': 0, 'max_tokens': 100, 'stop': '\n'}


TypeError: 'module' object is not subscriptable

In [12]:
hotpot = joblib.load('data/hotpot-qa-distractor-sample.joblib').reset_index(drop = True)

hotpot['supporting_paragraphs'] = None
for ind, row in hotpot.iterrows():
    supporting_articles = row['supporting_facts']['title']
    articles = row['context']['title']
    sentences = row['context']['sentences']
    supporting_paragraphs = []
    for article in supporting_articles:
        supporting_paragraph = ''.join(sentences[np.where(articles == article)][0])
        supporting_paragraphs.append(supporting_paragraph)
    supporting_paragraphs = '\n\n'.join(supporting_paragraphs)
    hotpot.at[ind, 'supporting_paragraphs'] = supporting_paragraphs

In [13]:
strategy: ReflexionStrategy = ReflexionStrategy.REFLEXION
from prompts import cot_agent_prompt, cot_reflect_agent_prompt, cot_reflect_prompt
from fewshots import COT, COT_REFLECT
agents = [CoTAgent(row['question'],
                   row['supporting_paragraphs'],
                   row['answer'],
                   agent_prompt=cot_agent_prompt if strategy == ReflexionStrategy.NONE else cot_reflect_agent_prompt,
                   cot_examples=COT,
                   reflect_prompt=cot_reflect_prompt,
                   reflect_examples=COT_REFLECT,
                    ) for _, row in hotpot.iterrows()]

In [17]:
agents[0].action_llm("Hello")

KeyboardInterrupt: 

In [12]:
n = 1
trial = 0
log = ''
for i in range(n):
    for agent in [a for a in agents[10:] if not a.is_correct()]:
        print(f"Question: {agent.question}")
        agent.run(reflexion_strategy = strategy)
        print(f'Answer: {agent.key}')
    trial += 1
    log += log_trial(agents, trial)
    correct, incorrect = summarize_trial(agents)
    print(f'Finished Trial {trial}, Correct: {len(correct)}, Incorrect: {len(incorrect)}')

Question: Stronger Together was used for the campaign comprised of the Vice Presidential candidate who was a Senator from what state?


KeyboardInterrupt: 